In [2]:
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Point

file_path = '/Users/yujinlee/Google_Drive/NYU/Fall_2023/Capstone/CrimeData'
df = pd.read_csv(file_path + '/la_2010_to_2019.csv', low_memory = False)

# df.to_csv('my_file.csv', index=False)

In [3]:
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [11]:
df.columns = map(str.lower, df.columns)

In [13]:
df.columns = df.columns.str.replace(' ', '_')

In [14]:
df.head()

,dr_no,date_rptd,date_occ,time_occ,area_,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,...,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [21]:
df['date_occ'] = df['date_occ'].str.slice(0, 10)
# Convert the 'occurred_on_date' column to datetime
df['date_occ'] = pd.to_datetime(df['date_occ'], format='%m/%d/%Y')

# Format the 'occurred_on_date' column to the desired format
df['date_occ'] = df['date_occ'].dt.strftime('%Y-%m-%d')

In [24]:
df['date_occ'] = pd.to_datetime(df['date_occ'])
threshold_date = pd.to_datetime('2018-01-01')
df = df[df['date_occ'] >= threshold_date]

In [30]:
df.to_csv(file_path + '/la_2010_to_2019.csv', index=False)

In [44]:
df.head()

,dr_no,date_rptd,date_occ,time_occ,area_,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,...,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
802737,212016382,11/01/2021 12:00:00 AM,2021-10-31,2300,20,Olympic,2029,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,"(34.0677, -118.285)",NaN,34.0677,-118.2850
802850,221904002,01/01/2022 12:00:00 AM,2021-12-31,1600,19,Mission,1987,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,"(34.2282, -118.4445)",NaN,34.2282,-118.4445
802906,221312198,06/02/2022 12:00:00 AM,2021-08-31,1200,13,Newton,1351,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,"(34.0038, -118.2805)",GRAND,34.0038,-118.2805
803605,211612176,10/05/2021 12:00:00 AM,2021-10-04,2030,16,Foothill,1612,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,NaN,NaN,NaN,"(34.2749, -118.4246)",BRADLEY,34.2749,-118.4246
804822,230606221,02/22/2023 12:00:00 AM,2021-09-11,1500,6,Hollywood,669,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,IC,Invest Cont,440.0,NaN,NaN,NaN,"(34.0908, -118.3025)",NaN,34.0908,-118.3025


In [32]:
df['lat'].isnull().mean()

0.0

In [34]:
df['lon'].isnull().mean()

0.0

In [35]:
df['location'] = df.apply(lambda row: f"({row['lat']}, {row['lon']})", axis=1)

In [36]:
file_path_geo = '/Users/yujinlee/Google_Drive/NYU/Fall_2023/Capstone/CrimeData/ZIP_Codes.geojson'

zipcode = gpd.read_file(file_path_geo)
zipcode.head()

802737     (34.0677, -118.285)
802850    (34.2282, -118.4445)
802906    (34.0038, -118.2805)
803605    (34.2749, -118.4246)
804822    (34.0908, -118.3025)
Name: location, dtype: object

In [39]:
file_path_geo = '/Users/yujinlee/Google_Drive/NYU/Fall_2023/Capstone/CrimeData/ca_california_zip_codes_geo.min.json'

zipcode = gpd.read_file(file_path_geo)
zipcode.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,06,94601,0694601,B5,G6350,S,8410939,310703,+37.7755447,-122.2187049,N,"POLYGON ((-122.22717 37.79197, -122.22693 37.7..."
1,06,94501,0694501,B5,G6350,S,20539466,9005303,+37.7737968,-122.2781230,N,"POLYGON ((-122.29181 37.76301, -122.30661 37.7..."
2,06,94560,0694560,B5,G6350,S,35757865,60530,+37.5041413,-122.0323587,N,"POLYGON ((-122.05499 37.54959, -122.05441 37.5..."
3,06,94587,0694587,B5,G6350,S,51075108,0,+37.6031556,-122.0186382,N,"POLYGON ((-122.06515 37.60485, -122.06499 37.6..."
4,06,94580,0694580,B5,G6350,S,8929836,17052,+37.6757312,-122.1330170,N,"POLYGON ((-122.12999 37.68445, -122.12995 37.6..."


In [47]:
def create_point_from_string(coord_str):
    match = re.match(r'\(([^,]+), ([^)]+)\)', coord_str)
    if match:
        lat, lon = map(float, match.groups())
        return Point(lon, lat)
    return None

In [48]:
df['geometry'] = df['location'].apply(create_point_from_string)

In [49]:
df.head()

,dr_no,date_rptd,date_occ,time_occ,area_,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,...,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon,geometry
802737,212016382,11/01/2021 12:00:00 AM,2021-10-31,2300,20,Olympic,2029,1,510,VEHICLE - STOLEN,...,Invest Cont,510.0,NaN,NaN,NaN,"(34.0677, -118.285)",NaN,34.0677,-118.2850,POINT (-118.285 34.0677)
802850,221904002,01/01/2022 12:00:00 AM,2021-12-31,1600,19,Mission,1987,1,510,VEHICLE - STOLEN,...,Invest Cont,510.0,NaN,NaN,NaN,"(34.2282, -118.4445)",NaN,34.2282,-118.4445,POINT (-118.4445 34.2282)
802906,221312198,06/02/2022 12:00:00 AM,2021-08-31,1200,13,Newton,1351,1,510,VEHICLE - STOLEN,...,Invest Cont,510.0,NaN,NaN,NaN,"(34.0038, -118.2805)",GRAND,34.0038,-118.2805,POINT (-118.2805 34.0038)
803605,211612176,10/05/2021 12:00:00 AM,2021-10-04,2030,16,Foothill,1612,1,330,BURGLARY FROM VEHICLE,...,Invest Cont,330.0,NaN,NaN,NaN,"(34.2749, -118.4246)",BRADLEY,34.2749,-118.4246,POINT (-118.4246 34.2749)
804822,230606221,02/22/2023 12:00:00 AM,2021-09-11,1500,6,Hollywood,669,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,Invest Cont,440.0,NaN,NaN,NaN,"(34.0908, -118.3025)",NaN,34.0908,-118.3025,POINT (-118.3025 34.0908)


In [50]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.head()

,dr_no,date_rptd,date_occ,time_occ,area_,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,...,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon,geometry
802737,212016382,11/01/2021 12:00:00 AM,2021-10-31,2300,20,Olympic,2029,1,510,VEHICLE - STOLEN,...,Invest Cont,510.0,NaN,NaN,NaN,"(34.0677, -118.285)",NaN,34.0677,-118.2850,POINT (-118.28500 34.06770)
802850,221904002,01/01/2022 12:00:00 AM,2021-12-31,1600,19,Mission,1987,1,510,VEHICLE - STOLEN,...,Invest Cont,510.0,NaN,NaN,NaN,"(34.2282, -118.4445)",NaN,34.2282,-118.4445,POINT (-118.44450 34.22820)
802906,221312198,06/02/2022 12:00:00 AM,2021-08-31,1200,13,Newton,1351,1,510,VEHICLE - STOLEN,...,Invest Cont,510.0,NaN,NaN,NaN,"(34.0038, -118.2805)",GRAND,34.0038,-118.2805,POINT (-118.28050 34.00380)
803605,211612176,10/05/2021 12:00:00 AM,2021-10-04,2030,16,Foothill,1612,1,330,BURGLARY FROM VEHICLE,...,Invest Cont,330.0,NaN,NaN,NaN,"(34.2749, -118.4246)",BRADLEY,34.2749,-118.4246,POINT (-118.42460 34.27490)
804822,230606221,02/22/2023 12:00:00 AM,2021-09-11,1500,6,Hollywood,669,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,Invest Cont,440.0,NaN,NaN,NaN,"(34.0908, -118.3025)",NaN,34.0908,-118.3025,POINT (-118.30250 34.09080)


In [51]:
gdf.set_crs(epsg=4326, inplace=True)
zipcode.set_crs(epsg=4326, inplace=True)
result = gpd.sjoin(gdf, zipcode, how="left", op='within')

/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [52]:
result.head()

,dr_no,date_rptd,date_occ,time_occ,area_,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,...,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
802737,212016382,11/01/2021 12:00:00 AM,2021-10-31,2300,20,Olympic,2029,1,510,VEHICLE - STOLEN,...,90020,0690020,B5,G6350,S,2927480.0,0.0,+34.0663795,-118.3098700,N
802850,221904002,01/01/2022 12:00:00 AM,2021-12-31,1600,19,Mission,1987,1,510,VEHICLE - STOLEN,...,91402,0691402,B5,G6350,S,9652884.0,77471.0,+34.2225061,-118.4446893,N
802906,221312198,06/02/2022 12:00:00 AM,2021-08-31,1200,13,Newton,1351,1,510,VEHICLE - STOLEN,...,90037,0690037,B5,G6350,S,7351184.0,0.0,+34.0027226,-118.2874866,N
803605,211612176,10/05/2021 12:00:00 AM,2021-10-04,2030,16,Foothill,1612,1,330,BURGLARY FROM VEHICLE,...,91331,0691331,B5,G6350,S,22928415.0,120343.0,+34.2554415,-118.4213135,N
804822,230606221,02/22/2023 12:00:00 AM,2021-09-11,1500,6,Hollywood,669,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,90029,0690029,B5,G6350,S,3526895.0,0.0,+34.0898482,-118.2946605,N


In [ ]:
result.set_index('date_occ', inplace=True)

In [55]:
result.index = pd.to_datetime(result.index)
result.head()

,dr_no,date_rptd,time_occ,area_,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,mocodes,...,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
date_occ,,,,,,,,,,,,,,,,,,,,,
2021-10-31,212016382,11/01/2021 12:00:00 AM,2300,20,Olympic,2029,1,510,VEHICLE - STOLEN,NaN,...,90020,0690020,B5,G6350,S,2927480.0,0.0,+34.0663795,-118.3098700,N
2021-12-31,221904002,01/01/2022 12:00:00 AM,1600,19,Mission,1987,1,510,VEHICLE - STOLEN,NaN,...,91402,0691402,B5,G6350,S,9652884.0,77471.0,+34.2225061,-118.4446893,N
2021-08-31,221312198,06/02/2022 12:00:00 AM,1200,13,Newton,1351,1,510,VEHICLE - STOLEN,NaN,...,90037,0690037,B5,G6350,S,7351184.0,0.0,+34.0027226,-118.2874866,N
2021-10-04,211612176,10/05/2021 12:00:00 AM,2030,16,Foothill,1612,1,330,BURGLARY FROM VEHICLE,0344,...,91331,0691331,B5,G6350,S,22928415.0,120343.0,+34.2554415,-118.4213135,N
2021-09-11,230606221,02/22/2023 12:00:00 AM,1500,6,Hollywood,669,1,440,THEFT PLAIN - PETTY ($950 & UNDER),1822 0344,...,90029,0690029,B5,G6350,S,3526895.0,0.0,+34.0898482,-118.2946605,N


In [ ]:
weekly_data = result.groupby('ZCTA5CE10').resample('W').size().reset_index(name='total_count')

In [58]:
weekly_data['ZCTA5CE10'].unique().shape

(148,)

In [60]:
weekly_data

,ZCTA5CE10,date_occ,total_count
0,90001,2018-01-07,21
1,90001,2018-01-14,14
2,90001,2018-01-21,17
3,90001,2018-01-28,16
4,90001,2018-02-04,23
...,...,...,...
27655,91801,2019-09-08,0
27656,91801,2019-09-15,0
27657,91801,2019-09-22,0
27658,91801,2019-09-29,1


In [63]:
mean_counts_by_zip = weekly_data.groupby('ZCTA5CE10')['total_count'].mean().reset_index()
mean_counts_by_zip

,ZCTA5CE10,total_count
0,90001,9.244019
1,90002,34.373206
2,90003,67.177033
3,90004,44.851675
4,90005,30.617225
...,...,...
143,91606,19.933014
144,91607,12.492823
145,91608,0.410000
146,91801,0.122222
